<a href="https://colab.research.google.com/github/sljm12/machine_learning_notebooks/blob/master/LLM/Transcript_Q%26A_with_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Youtube Transcript Q&A

Using the national day rally speech from https://www.youtube.com/watch?v=LCcWWbx6pXU

Prototype of a Youtube video Q&A with the ability to show the video segments in which the data is taken from from.


In [1]:
!pip install yt-dlp auto-gptq huggingface_hub sentencepiece chromadb langchain pypdf sentence_transformers tqdm gradio>/dev/null

## Clean VTT with timing

This is for cleaning the VTT file from yt-dlp the VTT file is full of duplicate lines so what we are scanning for are the duplicate lines and also removing all the ums, uhs from the text so that we have a cleaner text to work with

In [2]:
def clean_with_timing(filepath):
  data = ""
  prev_timing = ""
  prev_text = ""
  with open(filepath) as f:
      lines = f.readlines()
      prev=""
      for l in lines:
        # Removes all the um, uhs etc from the text
        for x in ['um','uh']:
          l=l.replace(x,"")
        if "<c>" in l: #This is one of the duplicate lines
            pass
        elif "-->" in l: #Save the timing info to see
            prev_timing=l
        elif len(l.strip()) == 0:
            pass
        else:
            if l.strip() != prev_text:
                prev_text=l.strip()
                data = data + prev_timing
                data = data + l
  return data

# Loading the Model

## Cloning the Vicuna model from TheBloke


In [3]:
#model_dir = "/content/vicuna-7B-v1.5-GPTQ"
model_dir = "/content/vicuna-13B-v1.5-16K-GPTQ"

In [4]:
from huggingface_hub import snapshot_download

snapshot_download(repo_id="TheBloke/vicuna-13B-v1.5-16K-GPTQ", local_dir=model_dir)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

'/content/vicuna-13B-v1.5-16K-GPTQ'

## Loading the model and tokenizer

In [5]:
from transformers import AutoTokenizer, TextGenerationPipeline
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

quantize_config = BaseQuantizeConfig(
        bits=4,
        group_size=128,
        desc_act=False
    )

tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model= AutoGPTQForCausalLM.from_quantized(model_dir, device="cuda:0",use_safetensors=True,use_triton=False)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Vicuna 7B ~ 5GB
Vicuna 13B 14GB

In [6]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!nvidia-smi

Tue Sep 12 00:46:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    29W /  70W |   8931MiB / 15360MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Setting up the text generation pipeline

In [7]:
RETURN_FULL_TEXT = False #If you set this to true it will return the prompt text also
pipeline = TextGenerationPipeline(model=model, tokenizer=tokenizer,max_new_tokens=512, return_full_text=RETURN_FULL_TEXT)

The model 'LlamaGPTQForCausalLM' is not supported for . Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMH

In [8]:
VICUNA_TEMPLATE ="""
A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

USER:

{prompt}

ASSISTANT:

"""

In [9]:
from langchain.chains.retrieval_qa.prompt import PROMPT, prompt_template

QA_PROMPT = VICUNA_TEMPLATE.format(prompt=prompt_template)

In [10]:
prompt_template="""
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
"""
QA_PROMPT = VICUNA_TEMPLATE.format(prompt=prompt_template)

In [11]:
print(QA_PROMPT)


A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

USER:


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}


ASSISTANT:




# Setting up ChromaDB imports and sentence embeddings

In [12]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Some time functions to extract the min max time from the VTT

This is for saving the min max time in the metadata

In [14]:
def get_time(text):
  segments = text.split(" --> ")
  return (segments[0], segments[1].split(" align:")[0])

def extract_min_max_time(text):
  first=""
  last=""
  lines = text.split("\n")
  for t in lines:
    if len(first) == 0 and "->" in t:
      first = t

    if "->" in t:
      last = t
  return get_time(first)[0],get_time(last)[1]

In [15]:
def convert_text_time_to_seconds(text):
  segments = [float(i) for i in text.split(":")]
  return segments[0]*60*60+segments[1]*60+segments[2]

In [16]:
print(convert_text_time_to_seconds("01:12:08.209"),convert_text_time_to_seconds("01:12:29.640"))

4328.209 4349.64


In [17]:
# Youtube player iframe
iframe_yt = """
<iframe id="ytplayer" type="text/html" width="640" height="360"
  src="{yt_url}"
  frameborder="0"></iframe>
"""

In [18]:
# The chromaDB variable we put it here so that even when the gradio app stops we can continue to query the DB
db = None

In [29]:
qs=urllib.parse.urlparse("https://www.youtube.com/watch?v=LCcWWbx6pXU").query
urllib.parse.parse_qs(qs)

{'v': ['LCcWWbx6pXU']}

In [35]:
os.remove("/content/data.txt.en.vtt")

In [32]:
db.similarity_search("What are the initiatives?")

[Document(page_content="we'll be continuing discussions we'll\n01:00:14.030 --> 01:00:14.040 align:start position:0%\nhave  follow webinars   hosted by\n01:00:15.470 --> 01:00:15.480 align:start position:0%\nalamadex on kind of talking about more\n01:00:16.789 --> 01:00:16.799 align:start position:0%\ncomponents of rag especially around\n01:00:19.010 --> 01:00:19.020 align:start position:0%\nstuff like observability evaluation and\n01:00:20.510 --> 01:00:20.520 align:start position:0%\nmaybe just a part two on a lot of these\n01:00:22.490 --> 01:00:22.500 align:start position:0%\nlike data decisions as well\n01:00:24.530 --> 01:00:24.540 align:start position:0%\n join us on on  follow us on Twitter", metadata={'end_time': 3624.54, 'source': '/content/data.txt', 'start_time': 3614.03, 'url': 'https://www.youtube.com/watch?v=LCcWWbx6pXU'}),
 Document(page_content="but for example getting getting also API\n00:32:56.210 --> 00:32:56.220 align:start position:0%\naccess to Google's Drive API

# The Gradio app

In [ ]:
import gradio as gr, os, glob, subprocess,time, datetime,urllib.parse


commandline = "yt-dlp --write-auto-sub --sub-langs \"en\" --skip-download --sub-format vtt {url} -o /content/data.txt"

def query(question):
  global db
  ans=db.similarity_search_with_score(question)
  context="\n".join([i[0].page_content+'\n' for i in ans])
  return pipeline(QA_PROMPT.format(question=question, context=context)), ans

def load_db(url):
  global db
  if db is not None:
    db.delete_collection()

  loader = DirectoryLoader('/content/', glob="*.txt", show_progress=True,loader_cls=TextLoader)
  loader = TextLoader("/content/data.txt")
  documents = loader.load_and_split()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=200)
  texts = text_splitter.split_documents(documents)

  #set metadata into the documents
  for t in texts:
    a=t.metadata
    a["url"]=url
    start,end=extract_min_max_time(t.page_content)
    a["start_time"]=convert_text_time_to_seconds(start)
    a["end_time"]=convert_text_time_to_seconds(end)
  db = Chroma.from_documents(texts, embeddings,collection_metadata={"url":url})


def download(u):
  global db
  s=commandline.format(url=u)
  os.system(s)

  time.sleep(5)
  vtt_files=glob.glob("*.vtt", root_dir="/content")
  #tb=vtt_files
  output = clean_with_timing("/content/"+vtt_files[0])
  with open("/content/data.txt","w") as f:
    f.write(output)
    load_db(u)
  #Clean up the left over files
  for i in vtt_files:
    os.remove("/content/"+i)
  os.remove("/content/data.txt")
  return "Done:"+datetime.datetime.now().isoformat() + " Collection Count:" + str(db._collection.count())

def extract_embed_url(link,start_time,end_time):
  qs=urllib.parse.urlparse(link).query
  params=urllib.parse.parse_qs(qs)
  return "https://youtube.com/embed/{id}?start={start}&end={end}".format(id=params["v"],
                                                                         start=int(start_time),
                                                                         end=int(end_time))

def return_html_iframe(links):
  return "\n".join([iframe_yt.format(yt_url=i) for i in links])

def query_click(query):
  global db
  ans=db.similarity_search_with_score(query)
  context="\n".join([i[0].page_content+'\n' for i in ans])
  video_ids = [extract_embed_url(i[0].metadata["url"],i[0].metadata["start_time"],i[0].metadata["end_time"]) for i in ans]
  return pipeline(QA_PROMPT.format(question=query, context=context))[0]["generated_text"],return_html_iframe(video_ids)

with gr.Blocks() as demo:
  youtube=gr.Textbox(label="Youtube URL", value ="https://www.youtube.com/watch?v=LCcWWbx6pXU")
  youtube_output=gr.Textbox(label="Loading")
  load_youtube=gr.Button(value="Load Youtube")
  load_youtube.click(download,inputs=[youtube],outputs=[youtube_output])

  query_text=gr.Textbox(label="Query",value="What are the initiatives in the rally?")
  output=gr.Textbox(lines=7,label="Output")
  html_1=gr.HTML()
  query_btn=gr.Button(value="Submit Query")
  query_btn.click(query_click,inputs=[query_text],outputs=[output,html_1])

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5af31015f983de669d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
